In [73]:
# Importing necessary libraries and importing CSV files into respective dataframes

import pandas as pd

df_CPI = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/CPI-inflation-by-region-1914-202.csv")
df_Housing = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/housing-supply-price-rental.csv")
df_HPIRegions = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 by regions.csv")
df_HPIFederalandProvincial = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 federal and provincial.csv")
df_Income = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/income-distribution-2012-2020.csv")
df_Interest = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Interest and mortgage rates 1951-2022.csv")
df_PopulationDwellings = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population_dwellings_count.csv")
df_PopulationRegion = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population-by-region-1946-2022.csv")
df_Structural = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Structural-dwellings-household-size.csv")
df_CPI['GEO'].unique()


array(['Canada', "St. John's, Newfoundland and Labrador",
       'Halifax, Nova Scotia', 'Saint John, New Brunswick',
       'Québec, Quebec', 'Montréal, Quebec',
       'Ottawa-Gatineau, Ontario part, Ontario/Quebec',
       'Toronto, Ontario', 'Thunder Bay, Ontario', 'Winnipeg, Manitoba',
       'Regina, Saskatchewan', 'Saskatoon, Saskatchewan',
       'Edmonton, Alberta', 'Calgary, Alberta',
       'Vancouver, British Columbia',
       'Charlottetown and Summerside, Prince Edward Island',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Whitehorse, Yukon',
       'Yellowknife, Northwest Territories', 'Victoria, British Columbia',
       'Iqaluit, Nunavut'], dtype=object)

In [74]:
# Removes region rows that aren't Canada and selects "All-items"
# df_CPI = df_CPI[df_CPI['GEO'] == 'Canada']
# df_CPI.drop(columns=['GEO'], inplace=True)

provinces = ['Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta']

df_CPI = df_CPI[df_CPI['GEO'].isin(provinces)]


df_CPI = df_CPI[df_CPI['Products and product groups'] == 'All-items']
df_CPI.drop(columns=['Products and product groups'], inplace=True)

# Removes unnecessary columns
df_CPI.drop(columns=['DGUID'], inplace=True)
df_CPI.drop(columns=['UOM'], inplace=True)
df_CPI.drop(columns=['UOM_ID'], inplace=True)
df_CPI.drop(columns=['VECTOR'], inplace=True)
df_CPI.drop(columns=['COORDINATE'], inplace=True)

# Standardizes "Date" name and format
df_CPI['Date'] = df_CPI['REF_DATE'].astype(str) + '01'
df_CPI.drop(columns=['REF_DATE'], inplace=True)

# df_CPI.head(108)

In [75]:
# Standardizes "Date" format and name
df_Housing['Date'] = df_Housing['year'].astype(str).str.replace('.0', '').str.replace('.1', '') + '01'

df_Housing = df_Housing[['Date', 'region', 'population', 'total_dwelling', 'total_dwelling_market', 'labour_participation_rate', 'unemployment_rate', 'disposable_income_change', 
                         'rental_vacancy_rate', 'rental_avilability_rate', 'owned_accommodation_costs_change', 'rental_accommodation_costs_change']]

regions_to_include = ['manitoba', 'prince_edward', 'new_brunswick', 'saskatchewan', 'nova_scotia', 'quebec', 'alberta', 'ontario']

df_Housing = df_Housing[df_Housing['region'].isin(regions_to_include)]

province_mapping = {
    'manitoba': 'Manitoba',
    'prince_edward': 'Prince Edward Island',
    'new_brunswick': 'New Brunswick',
    'saskatchewan': 'Saskatchewan',
    'nova_scotia': 'Nova Scotia',
    'quebec': 'Quebec',
    'alberta': 'Alberta',
    'ontario': 'Ontario'
}

# Update province names in the DataFrame
df_Housing['region'] = df_Housing['region'].map(province_mapping)

df_Housing.shape

(216, 12)

In [76]:

df_HPIRegions = df_HPIRegions[df_HPIRegions['Type'] == 'House and Land']
df_HPIRegions.drop(columns = ['Type'], inplace=True)

# List of columns representing regions
regions_columns = ['Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta']

# Melt the dataframe to transform the columns into a single 'region' column
df_HPIRegions = df_HPIRegions.melt(id_vars=['Month-year'], value_vars=regions_columns, var_name='region', value_name='value')
df_HPIRegions = df_HPIRegions.dropna()
df_HPIRegions.head(20)



# Converts "Month-year" to standard format and renames column to "Date"
df_HPIRegions['Month-year'] = pd.to_datetime(df_HPIRegions['Month-year'], format='%b-%y')
def adjust_year(date):
    
    if date.year - 2000 >= 25 and date.year - 2000 <= 99:
        return date.replace(year=date.year - 100)

    return date
df_HPIRegions['Month-year'] = df_HPIRegions['Month-year'].apply(adjust_year)
df_HPIRegions['Month-year'] = df_HPIRegions['Month-year'].dt.strftime('%Y%m')
df_HPIRegions.rename(columns={'Month-year': 'Date'}, inplace=True)
# df_HPIRegions.rename(columns={'Month-year': 'Date'}, inplace=True)

df_HPIRegions.head(1494)

,Date,region,value
60,198601,Prince Edward Island,83.5
61,198602,Prince Edward Island,83.5
62,198603,Prince Edward Island,83.5
63,198604,Prince Edward Island,83.5
64,198605,Prince Edward Island,83.5
...,...,...,...
1729,200008,Quebec,58.3
1730,200009,Quebec,58.6
1731,200010,Quebec,58.8
1732,200011,Quebec,59.0


In [77]:
df_Housing.shape

(216, 12)

In [78]:
merged_df = pd.merge(df_Housing, df_HPIRegions, left_on=['Date', 'region'], right_on=['Date', 'region'], how='left')
merged_df = pd.merge(merged_df, df_CPI, left_on=['Date', 'region'], right_on=['Date', 'GEO'])
merged_df = merged_df.dropna()
merged_df.drop(columns=['GEO'], inplace=True)
merged_df.rename(columns={'value': 'HPI'}, inplace=True)

# Calculate the number of months passed
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%Y%m')
# merged_df['Num of Months Passed'] = (merged_df['Date'] - merged_df['Date'].min()) // pd.Timedelta(days=30)
merged_df['year'] = merged_df['Date'].dt.year

merged_df.shape

(216, 15)

In [79]:
provinces = pd.get_dummies(merged_df['region'], prefix='region')
provinces = provinces.astype(int)
merged_df = merged_df.join(provinces)
merged_df = merged_df.drop(['region'], axis=1) 
merged_df.head()

,Date,population,total_dwelling,total_dwelling_market,labour_participation_rate,unemployment_rate,disposable_income_change,rental_vacancy_rate,rental_avilability_rate,owned_accommodation_costs_change,...,CPI,year,region_Alberta,region_Manitoba,region_New Brunswick,region_Nova Scotia,region_Ontario,region_Prince Edward Island,region_Quebec,region_Saskatchewan
0,1990-01-01,1105.421,3297,2274,67.300,7.4,-2.669133,6.1,2.7,4.577354,...,76.2,1990,0,1,0,0,0,0,0,0
1,1991-01-01,1109.604,1950,1438,67.025,8.6,-2.669133,6.2,2.7,5.829327,...,80.1,1991,0,1,0,0,0,0,0,0
2,1992-01-01,1112.689,2310,1821,66.475,9.3,-0.326989,5.8,2.7,0.359644,...,81.2,1992,0,1,0,0,0,0,0,0
3,1993-01-01,1117.618,2425,1702,66.850,9.3,-0.453243,5.6,2.7,-0.028291,...,83.4,1993,0,1,0,0,0,0,0,0
4,1994-01-01,1123.230,3197,1664,66.750,8.8,0.161821,5.5,2.7,1.480992,...,84.6,1994,0,1,0,0,0,0,0,0


In [80]:
merged_df.head()

,Date,population,total_dwelling,total_dwelling_market,labour_participation_rate,unemployment_rate,disposable_income_change,rental_vacancy_rate,rental_avilability_rate,owned_accommodation_costs_change,...,CPI,year,region_Alberta,region_Manitoba,region_New Brunswick,region_Nova Scotia,region_Ontario,region_Prince Edward Island,region_Quebec,region_Saskatchewan
0,1990-01-01,1105.421,3297,2274,67.300,7.4,-2.669133,6.1,2.7,4.577354,...,76.2,1990,0,1,0,0,0,0,0,0
1,1991-01-01,1109.604,1950,1438,67.025,8.6,-2.669133,6.2,2.7,5.829327,...,80.1,1991,0,1,0,0,0,0,0,0
2,1992-01-01,1112.689,2310,1821,66.475,9.3,-0.326989,5.8,2.7,0.359644,...,81.2,1992,0,1,0,0,0,0,0,0
3,1993-01-01,1117.618,2425,1702,66.850,9.3,-0.453243,5.6,2.7,-0.028291,...,83.4,1993,0,1,0,0,0,0,0,0
4,1994-01-01,1123.230,3197,1664,66.750,8.8,0.161821,5.5,2.7,1.480992,...,84.6,1994,0,1,0,0,0,0,0,0


## Model and Feature Selection

### Base model

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics


y= merged_df['HPI']
# dropped date until we use XGBoost
X = merged_df.drop(['HPI', 'Date'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, stratify=merged_df['year'])

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 55.91041733072887
Root Mean Squared Error (RMSE): 7.47732688403609
R-squared (R2): 0.8832448616097297


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Model using Backwards Stepwise selection

In [82]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

model = LinearRegression()

selector = SequentialFeatureSelector(model, direction='backward')

selector.fit(X_train, y_train)

selected_features = X_train.columns[selector.get_support()]

model.fit(X_train[selected_features], y_train)

y_pred = model.predict(X_test[selected_features])

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)

print("Selected Features:", selected_features)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Selected Features: Index(['total_dwelling', 'unemployment_rate', 'rental_avilability_rate',
       'rental_accommodation_costs_change', 'CPI', 'region_Alberta',
       'region_New Brunswick', 'region_Nova Scotia', 'region_Ontario',
       'region_Prince Edward Island'],
      dtype='object')
Mean Squared Error (MSE): 58.3483411380683
Root Mean Squared Error (RMSE): 7.638608586520735
R-squared (R2): 0.8781538580883803


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [83]:
from sklearn.linear_model import Lasso


model = Lasso(alpha=0.1)  

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 55.57717050867546
Root Mean Squared Error (RMSE): 7.455009759126775
R-squared (R2): 0.8839407655339806


/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.074e+03, tolerance: 7.667e+00
  model = cd_fast.enet_coordinate_descent(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
